In [4]:
# Load the required packages
import time
import psutil
import numpy as np
import pandas as pd
import glob
import multiprocessing as mp

# For data visualizatoin
import matplotlib.pyplot as plt
from pandas.tools.plotting import autocorrelation_plot
from bokeh.io import output_notebook, show
from bokeh.models import Title
from bokeh.plotting import figure, output_file, show

from datetime import datetime, timedelta

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import xgboost as xgb


In [2]:
## GET THE DATA FROM FILES

class data(object):
    
    def __init__(self):        
        self.df_aggregate = pd.DataFrame()
    
    def get_consumption(start_date, end_date):
#         path=r'./UKDA-7857-csv/csv/data_collection/data_tables'
#         files_= glob.glob(path+"/*.csv")
        df_consumption_d = pd.read_csv('./UKDA-7857-csv/csv/data_collection/data_tables/consumption_d.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
        df_consumption_n = pd.read_csv('./UKDA-7857-csv/csv/data_collection/data_tables/consumption_n.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
        df_tariff = pd.read_csv('./UKDA-7857-csv/csv/data_collection/data_tables/tariff_d.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
        
        df_trial_d = df_consumption_d.loc['2013-01':'2013-12',:]
        df_trial_n = df_consumption_n.loc[start_date:end_date,:]
        
        self.df_aggregate['GMT'] = df_trial_n.index
        self.df_aggregate=df_aggregate.set_index('GMT')
        
        self.df_aggregate['trial_n']=df_trial_n.mean(axis=1)
        self.df_aggregate['trial_d']=df_trial_d.mean(axis=1)

        # Take aggregate of the users
        self.df_aggregate['trial_d']=df_trial_d.mean(axis=1)
        self.df_aggregate['trial_n']=df_trial_n.mean(axis=1)
        
        # Add tariff details
        self.df_aggregate['tariff']=df_tariff.loc['2013-01':'2013-12','Price']
        
    
    def get_weather(start_date, end_date):
        df_midas=pd.read_csv('./mod_datasets/midas_weather.csv', sep=',', header=0, index_col=0, parse_dates=['GMT'], low_memory=False)
        df_midas_rs = df_midas.resample('30T').mean()
        df_interpolated = df_midas_rs.interpolate(method='linear')
        df_weather = df_interpolated.loc[start_date:end_date,:]
        
        self.df_aggregate = pd.concat([self.df_aggregate, df_weather], axis = 1)
        
    
    def get_survey(question_no):
        df_questions = pd.read_csv(files_[3], sep=',', header=0, index_col=0, low_memory=False)
        df_answers = pd.read_csv(files_[1], sep=',', header=0, index_col=0, low_memory=False )
        # YET TO FINISH (updating after initial active learning results)
        
        


In [3]:
class Simulator(object):
    
    def __init__(self):        
        self.xg_reg = xgb.XGBRegressor(n_estimators=50)
        self.preds = []
    
    def train(self, df, X_variables, y_variable):
        X_train = df.loc['2013-01':'2013-09',X_variables]
        y_train = df.loc['2013-01':'2013-09',y_variable]
        X_test = df_all.loc['2013-10':,X_variables]
        y_test = df_all.loc['2013-10':,y_variable]
        
        # Fit the data
        self.xg_reg.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_test, y_test)],
                early_stopping_rounds=50, #stop if 50 consequent rounds without decrease of error
                verbose=False) # Change verbose to True if you want to see it train
    
        xgb.plot_importance(xg_reg, height=0.9) # Plot feature importance
        
        
    def get_label(self,df):
        self.preds = self.xg_reg.predict(X_test)
        
        
    
        

In [ ]:
class BaseModel(object):
    
    def __init__(self):
        pass
    
    def fit_predict(self):
        pass
    
    
class RFModel(BaseModel):
    
    def fit_predict(self, X_train, y_train, X_val, X_test, c_weight):
        self.classifier = RandomForestClassifier(n_estimators=500, class_weights = c_weight)
        self.classifier.fit(X_train, y_train)